In [ ]:
import numpy as np
from scipy.integrate import odeint
from scipy.optimize import curve_fit
from scipy.optimize import differential_evolution
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd
import sys
np.set_printoptions(threshold=sys.maxsize)

In [ ]:
#t = np.linspace(0,34800, num=6961)
t = np.linspace(0,61200, num=12241)
#time=t[0:34800:5]
rawdata = np.transpose(np.delete(np.genfromtxt('chz1longdata.csv',delimiter=','),0,0))
newdata = list(rawdata.flatten())

lightdata = np.transpose(np.delete(np.genfromtxt('longlight.csv', delimiter=','),0,0))
#param = np.linspace(0.00001, 1000, 10000)
#print(len(param))

def func(t):
    z0 = [1,0,0,0,0,0,0]
    #mCherry2 = np.empty((len(lightdata[:,0]), len(t)))
    arrayvalues = np.asarray([])
    out = np.zeros((len(lightdata[:,0]),len(t)))

    for i in range(2):
    #for i in range(len(lightdata[:,0])):
        def I(t):
            tindex = t/5
            if tindex > 12241:
                tindex = 12240
                #print(lightdata[i][int(tindex)])
            return lightdata[i][int(tindex)]
        
       # for j in range(len(param)):
            def model(z,t):
                #p1 = param;
                #CR specific parameters
                Kd = 90.41
                n = 0.964487

                d2 = 481.96
                k2 = 0.0854
                k3 = 0.000136
                k7 = 15.20

                #constant parameters
                Pu,Pb,Pa,mRNA,mCherry1,mCherry2,mCherry3 = z
                d1 = 0.019905
                k1 = 0.08299
                d3 = 0.000077
                k4 = 1.25
                d4 = 0.000031
                k5 = 0.00283
                k6 = 0.00283
                
                Pu = z[0]
                Pb = z[1]
                Pa = z[2]
                mRNA = z[3]
                mCherry1 = z[4]
                mCherry2 = z[5]
                mCherry3 = z[6]

                dPudt = d1*Pb - k1*I(t)**n/(Kd**n+I(t)**n)*Pu - k7*I(t)**n/(Kd**n+I(t)**n)*Pu
                dPbdt = k1*I(t)**n/(Kd**n+I(t)**n)*Pu - k2*Pb - d1*Pb + d2*Pa
                dPadt = k2*Pb - d2*Pa + k7*I(t)**n/(Kd**n+I(t)**n)*Pu
                dmRNAdt = k3*Pa - d3*mRNA
                dmCherry1dt = k4*mRNA-(d4 + k5)*mCherry1
                dmCherry2dt = k5*mCherry1-(d4+k6)*mCherry2
                dmCherry3dt = k6*mCherry2 - d4*mCherry3

 
                return [dPudt,dPbdt,dPadt,dmRNAdt,dmCherry1dt,dmCherry2dt,dmCherry3dt]  
                z = odeint(model,z0,t,hmax=0.1)
                out = z[:,6]
               # end[j,i] = mCherry3
            
    return out

In [ ]:
rawdata = np.transpose(np.delete(np.genfromtxt('chz1longdata.csv',delimiter=','),0,0))
#print(len(rawdata))
mCherry3 = np.asarray(model1)
mCherry3 = list(mCherry3.flatten())
exp_mCherry3 = mCherry3[0:12240:240]
#exp_mCherry3 = mCherry3[0:6960:240]
#print(exp_mCherry3)
print(len(exp_mCherry3))

exp_mCherry3 = np.array(exp_mCherry3)
print(exp_mCherry3)

ydata = np.asarray(rawdata)
ydata = (ydata.flatten())
exp_ydata=ydata[0:12240:240]
#exp_ydata=ydata[0:6960:240]

exp_ydata = np.array(exp_ydata)
#print(exp_ydata)
print(len(exp_ydata))

ssr = np.sum((exp_ydata-exp_mCherry3)**2)
#ssr2 = np.sum((ydata-model3)**2)
sst = np.sum((exp_ydata - np.mean(exp_ydata))**2)
R2 = 1 - ssr/sst
#R2_2 = 1 - ssr2/sst

print('R2 is: ', R2)

import math
from sklearn.metrics import mean_squared_error
mse=mean_squared_error(exp_ydata, exp_mCherry3)
print(mse)

num_params = 4
n = len(exp_ydata)
#print(n)

def calculate_aic(n,mse,num_pararms):
    aic = n*math.log(mse) + 2*num_params
    return aic
aic = calculate_aic(n,mse,num_params)
print(aic)

def calculate_bic(n, mse, num_params):
    bic = n*math.log(mse) + num_params*math.log(n)
    return bic
bic = calculate_bic(n,mse,num_params)
print(bic)

In [ ]:
for i in range(2):
#for i in range(len(lightdata[:,0])):
    mCherry3 = model1
    df = pd.DataFrame(mCherry3)
    df.to_csv("VP16_3stFBLB_mCherry3raw.csv")


In [ ]:
pp = PdfPages('multipage.pdf')

for j in range(len(lightdata[:,0])):
    plt.plot(t,model1[j],'*', label = 'model')
    plt.plot(t,rawdata,'.',label = 'data')
    plt. suptitle('Test set',fontsize = 14)
    plt.ylabel('Fluoresence [a.u.]')
    plt.xlabel('Condition')
    plt.legend()
    pp.savefig()
    plt.show()
pp.close()